In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pykrige.ok import OrdinaryKriging
import rasterio
from rasterio.transform import from_origin
from rasterio.mask import mask
from pathlib import Path
import os

GRID_RESOLUTION = 0.001
KABUPATEN_TARGET = "Purbalingga"
CRS_OUTPUT = 'EPSG:4326'

DATA_DIR = Path('data')
OUTPUT_DIR = Path('output')
PETA_DIR = DATA_DIR / 'peta_purbalingga'
KEJADIAN_FILE = DATA_DIR / 'kejadian.csv'
SHAPEFILE_KEC = PETA_DIR / 'gadm41_IDN_3.shp'
SHAPEFILE_DESA = PETA_DIR / 'gadm41_IDN_4.shp'

OUTPUT_RASTER_FILE = OUTPUT_DIR / 'peta_kerawanan_purbalingga.tif'
TEMP_RASTER_FILE = OUTPUT_DIR / 'temp_raster.tif'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("--- Proses Pemetaan Final dan Ekspor Raster Dimulai ---")
print("\nLangkah 1: Memuat semua data...")

try:
    df_crime = pd.read_csv(KEJADIAN_FILE)
    gdf_all_kecamatan = gpd.read_file(SHAPEFILE_KEC)
    gdf_all_desa = gpd.read_file(SHAPEFILE_DESA)
    print("✓ Data kejadian, kecamatan, dan desa berhasil dimuat.")
except Exception as e:
    print(f"❌ ERROR: Gagal memuat file. Detail: {e}")
    exit()

print(f"\nLangkah 2: Memfilter peta untuk wilayah {KABUPATEN_TARGET}...")
gdf_kec_pbg = gdf_all_kecamatan[gdf_all_kecamatan['NAME_2'] == KABUPATEN_TARGET]
gdf_desa_pbg = gdf_all_desa[gdf_all_desa['NAME_2'] == KABUPATEN_TARGET]

if gdf_kec_pbg.empty or gdf_desa_pbg.empty:
    print(f"❌ ERROR: Tidak ditemukan data untuk '{KABUPATEN_TARGET}'.")
    exit()
else:
    print(f"✓ Peta kecamatan dan desa berhasil difilter.")
    
print("\nLangkah 3: Mempersiapkan data untuk Kriging...")
lons = df_crime['longitude'].values
lats = df_crime['latitude'].values
values = df_crime['jumlah_kejadian'].values
print(f"✓ {len(values)} titik data siap untuk interpolasi.")

print("\nLangkah 4: Melakukan Interpolasi Kriging final...")
OK_final = OrdinaryKriging(
    lons, lats, values,
    variogram_model="spherical",
    verbose=False, enable_plotting=False
)

print("\nParameter Variogram (sill, range, nugget):")
print(OK_final.variogram_model_parameters)

min_lon, min_lat, max_lon, max_lat = gdf_desa_pbg.total_bounds
grid_lon = np.arange(min_lon, max_lon + GRID_RESOLUTION, GRID_RESOLUTION)
grid_lat = np.arange(min_lat, max_lat + GRID_RESOLUTION, GRID_RESOLUTION)

zdata, ss = OK_final.execute("grid", grid_lon, grid_lat)
print("✓ Interpolasi pada grid selesai.")

print("\nLangkah 5: Menyimpan hasil interpolasi ke file GeoTIFF (Temporary)...")
zdata_flipped = np.flip(zdata, axis=0)
transform = from_origin(min_lon, max_lat, GRID_RESOLUTION, GRID_RESOLUTION)

try:
    with rasterio.open(
        TEMP_RASTER_FILE, 'w', driver='GTiff',
        height=zdata_flipped.shape[0], width=zdata_flipped.shape[1],
        count=1, dtype=zdata_flipped.dtype, crs=CRS_OUTPUT, transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(zdata_flipped, 1)
    print(f"✓ Raster temporary berhasil dibuat di '{TEMP_RASTER_FILE}'")

    print("\nLangkah 6: Memotong raster sesuai batas wilayah Purbalingga...")
    with rasterio.open(TEMP_RASTER_FILE) as src:
        gdf_desa_pbg = gdf_desa_pbg.to_crs(src.crs)
        purbalingga_boundary_geoms = [geom for geom in gdf_desa_pbg.geometry]
        
        out_image, out_transform = mask(
            src, 
            purbalingga_boundary_geoms, 
            crop=True, 
            nodata=np.nan, 
            all_touched=True
        )
        out_meta = src.meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "nodata": np.nan
    })
    with rasterio.open(OUTPUT_RASTER_FILE, "w", **out_meta) as dest:
        dest.write(out_image[0], 1)

    print(f"✓ Raster final berhasil dipotong dan disimpan sebagai '{OUTPUT_RASTER_FILE}'")

    print("\n--- Proses Selesai ---")

except Exception as e:
    print(f"❌ ERROR pada Langkah 5 atau 6: {e}")

finally:
    if os.path.exists(TEMP_RASTER_FILE):
        os.remove(TEMP_RASTER_FILE)
        print(f"✓ File temporary '{TEMP_RASTER_FILE}' telah dihapus.")

--- Proses Pemetaan Final dan Ekspor Raster Dimulai ---

Langkah 1: Memuat semua data...
✓ Data kejadian, kecamatan, dan desa berhasil dimuat.

Langkah 2: Memfilter peta untuk wilayah Purbalingga...
✓ Peta kecamatan dan desa berhasil difilter.

Langkah 3: Mempersiapkan data untuk Kriging...
✓ 29 titik data siap untuk interpolasi.

Langkah 4: Melakukan Interpolasi Kriging final...

Parameter Variogram (sill, range, nugget):
[0.23447391 0.01835441 0.09537762]
✓ Interpolasi pada grid selesai.

Langkah 5: Menyimpan hasil interpolasi ke file GeoTIFF (Temporary)...
✓ Raster temporary berhasil dibuat di 'output\temp_raster.tif'

Langkah 6: Memotong raster sesuai batas wilayah Purbalingga...
✓ Raster final berhasil dipotong dan disimpan sebagai 'output\peta_kerawanan_purbalingga.tif'

--- Proses Selesai ---
✓ File temporary 'output\temp_raster.tif' telah dihapus.
